# Measuring grain sizes

In this notebook we will demonstrate how we can use the [scikit-image library](https://scikit-image.org/) to determine grain sizes from micrographs and how to use [matplotlib](https://matplotlib.org/) to visualize the results.

Concretely, we will look at:
* what is an image
* how to import images with [pillow](https://python-pillow.org/)
* how to make histograms from images
* how to threshold images and deciding on thresholds
* how to clean up images
* how to plot multiple images on top of eachother and create custom color maps
* how to segment different contiguous regions
* a bit more advanced segmentation with watershed
* measuring and plotting properties of segmented images
* exporting tables of properties

## Import some libraries and define settings

In [ ]:
%matplotlib inline

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["image.cmap"] = "Greys_r"

## What is an image?
An image is simply a 2D array of numbers.
Pixels are rows and columns, and the pixel intensity is represented by a number.

There are different ways to classify images:
* color
    * **grayscale** - one value per pixel
        * value corresponds linearly to pixel "intensity"
    * **color** - three values red, green, blue (RGB)
        * sometimes there is a 4th "channel" - alpha, to indicate translucency
* pixel depth
    * 8-bit: pixel values are integers ranging from 0 - 255 (2^8 - 1)
    * 16-bit: pixel values range from 0 - 65535 (2^16 - 1)
    * float: intensities can be any decimal that can be represented with 64 bits, usually scaled from 0-1
* file formats
    * there are many ways to store an image to disk (.png, .jpg, .tiff, ...)
    * they encode the same type of information but in a different way, often using compression
    * different file format may support different pixel depths
    * jpg: always 8-bit RGB, lossy compressed. Not good for images with sharp edges.
    * tiff: lossless storage, supports rich metadata
    
We can create some examples of images using numpy:

In [ ]:
# small grayscale image
grayscale = np.array([[0, 100, 255],
                      [100, 0,   0],
                      [50, 50,  255],
                      [255, 0,  255],
                     ], dtype=np.uint8)
print(grayscale.shape)
print(grayscale)
plt.imshow(grayscale)

Matplotlib doesn't care what kind of data is stored in our array. We can convert 8-bit to floats 0-1 by dividing.

### Exercise 1
* create your own grey-scale "image" array
* try to view some properties of your image
    * try to print out the maximum of the array
    * try to print out the minimum
    * try to print out the contained dtype
    * try to print out the shape
* experiment with some operations on the image like multiplying it by something. Can you multiply it by a number? Divide it by a number? Multiply it by itself?
* try to take the square root of your image
* can you visualize the image that has been operated on? Note that the coloring of a grayscale image is arbitrary. Can you plot the image with a different color map?

Matplotlib can also plot RGB images but then it expects values between 0-1 for red, green and blue. RGBA images can also be plot by adding a 4th value.

In [ ]:
# RGB image
np.random.seed(42)
# create a random (4x5) RGB image
rgb_image = np.random.rand(4, 5, 3)
plt.imshow(rgb_image)

## Importing images
For importing images an easy library to use is PIL. 
We can directly visualize the image in a jupyter notebook simply by typing the variable name.

We will analyze an image of grains to see whether we can separate the grains and measure their size. Do you already have some ideas how we could approach this?

In [ ]:
image = Image.open("./grain_image_2.jpg")
image

PIL imports the image as an image object, we can convert this in a very simple way to a numpy array and check the information inside.

### Exercise 2

Try to convert the image to a numpy array. Check the properties of the array (dtype, shape, maximum, minumum, ...). Can you plot this array with matplotlib?

We imported a jpg, so the data stored inside each pixel is an RGB value ranging from 0-255. Since color is not important in this image, we want to convert this to simple grayscale. 

### Exercise 3
Try to make an array corresponding to a greyscale image by taking the mean of each color channel. Verify that it looks ok by plotting it.

### Exercise 4
Generally to convert to grayscale we use a weighted sum of red, green and blue. The weights take into consideration our eye's sensitivity to different colors. 

* There is a function in scikit image that does this, can you find it with google?
* Use the function and plot the result. Do you see any differences between the regular mean and the weighted average?

Maybe what we can do as a first step is separate the grain boundaries from the grain interior using thresholding. The grain boundaries are clearly darker than the grain interiors, so we may be able to separate this easily by defining some cut-off value. Above the value, everything is grain, below everything is boundary. But where to we put this threshold?

### Excercise 5
A convenient tool to inspect the kinds of intensities we have in the image are histograms. With matplotlib this can be done in a very convenient way using the `hist` method.
* Try to make a histogram of the image intensities (tip: you may have to unravel the image first)
* Play with some of the options in the `hist` method and see how it changes the plot. Can you change the color? Can you change the number of bars?

The histogram tells us most of the pixel intensities center around 0.8. We can also see a lot of pixels are completely white while others are completely black. The best separation will likely be around 0.6, but how do we decide an optimal value? Luckily there are a number of strategies to determine an "optimal" threshold value and we can directly check all those implemented in scikit image using the `try_all_threshold` method. 

### Exercise 6
Try to use the `try_all_threshold` method and decide on a good threshold method

In [ ]:
from skimage.filters import try_all_threshold

Here it depends a bit on our own preferences whether we want black to cut into our grains or whether we want to miss some faint grain boundaries. 

### Exercise 7

* Select a thresholding method you find optimal and use it to find the threshold value. Print it out to check it.
* We then create a thresholded image by comparing the gray value image to the threshold value. This creates a "boolean" image, where each pixel is `True` (the pixel intensity is larger than threshold = grain interior) or `False` (the pixel intensity is smaller than threshold = boundary or inclusion). Can you create such a binary image? Check the dtype.
* Plot the boolean image

In [ ]:
from skimage.filters import # import the threshhold method you want

tresh = # use the threshold value

binary = # create a binary image

# plot the binary image

## Cleaning up the image

There are a lot of little black spots in our grains due to little inclusions. Fortunately it is quite easy to clean those up using the scikit-image morphology functions. In short, these can be used to expand or shrink black/white regions respectively according to some rules. 

### Exercise 8
* look at the documentation of functions in the `skimage.morphology` module. In particular explore functions lik `area_closing`, `erosion`, `dilation` and `area_opening`. What happens to the binary image when you use these functions on them? Which one might you use to remove some of the noise in the image?

In [ ]:
from skimage.morphology import # look at the documentation

cleaned = # use something to clean the binary image
# plot the result

## Plotting images on top of eachother and custom color maps
Let's check if our cleaned boolean image still matches ok with our original image by plotting them on top of eachother. Since our grains are white in our boolean image, we will not be able to see anything of the image plotted below. Let's create a custom RGBA color map that is transparent at 0 (=False) and opaque at 1 (=True).

### Exercise 9
* Create a (x, 4)-shaped array to represent a custom RGBA mapping of your own design. It might be best if the grain boundaries are colored and the grain interiors are transparant. Experiment with the value of x.
* Plot the original image and the cleaned grain boundaries

In [ ]:
# Create a (x, 4) array that serves as RGBA mapping

from matplotlib import colors
cmap = colors.ListedColormap(cmap_custom) # this creates our custom map

fig, ax = plt.subplots()
# use the ax object like plt beforehand for persistant plotting on the same image

## Segmenting contiguous regions.
In principle all we need is for the different connected white regions to be joined together as grains. So what we need to do is *label* the white pixels with a number that will indicate the "grain" it belongs to. This is best performed using scipy's ndimage module which contains the `label` method. Note that we define a "structure", which  defines how "contiguity" should be defined. If we say np.ones((3, 3)) we indicate that all 8 pixels around each pixel should be considered for contiguity. We could also limit contiguity more and say we don't want the corner elements. We could construct this matrix manually, but easier is using scikit image's `disk`.

In [ ]:
from skimage.morphology import disk

square = np.ones((3, 3))
selem = disk(1)
print("Consider diagonal elements contiguous:")
print(square)
print("Don't consider diagonal elements contiguous:")
print(selem)

In [ ]:
import scipy.ndimage as ndi

### Exercise 10

Use the `label` function in `ndimage` to create a labeled image.
* Can you find a use for the contiguity arrays from before?
* What is the structure of the output, can you plot it?

We have detected hundreds of grains, and they are colored from from top to bottom depending which cmap you used. It may be easier to detect the different grains by converting the labels to random RGB values, which can be done with `skimage.label2rgb`. 

*Note that some grains next to eachother may by chance still get the same color.*

In [ ]:
from skimage.color import label2rgb
colors = # use the label2rgb function to colorize the labeled image
plt.imshow(colors)

## More advanced segmentation with watershed

You can see that some grain boundaries have little gaps in them, which incorrectly connects some grains by contiguity. We can try to separate those out with the watershed algorithm. With watershed we will try to pinpoint the centers of grains and grow those outwards (as lakes growing) in a "landscape". The algorithm stops when the different "waterlines" touch.

First we need some "topology" map of the image. We will use the distance transform, which measures how far away each pixel is from the nearest black (background) pixel. This transform is again in the scipy.ndimage module.

In [ ]:
dist = ndi.distance_transform_edt(cleaned)

### Exercise 11
If we plot the negative distance, we can image each grain being kind of like a valley that needs to be filled up with water. Plot the positive and negative distance.

Now we still need to find the locations of the "fountains" from where we fill the landscape.
A good place to add fountains is at the bottom of the valleys; we can find these using a peak finding algorithm.

### Exercise 12
* In the following cell we find the local peaks in two ways. Explore what the output is and what the effect is of different parameters. 
* Can you plot the distance map with the coordinates of the fountains on top?

In [ ]:
from skimage.feature import peak_local_max
from skimage.filters import gaussian
# we smooth the distance map a bit to have less noise in peak finding
coords = peak_local_max(gaussian(dist, sigma=1.5), min_distance=5, threshold_rel=0.1, exclude_border=False)

# to get the fountain coordinates in an image which is supplied to watershed
mask = peak_local_max(gaussian(dist, sigma=1.5), min_distance=5, threshold_rel=0.1,
                      exclude_border=True, indices=False)
# We have to label the fountains each with a unique number
labels_water, _ = ndi.label(mask)

#plot the distance map with the peaks on top

Now we just provide the labeled mask (each non-zero pixel has a unique integer and corresponds to a fountain) to the watershed algorithm.

### Exercise 13
Try to use the watershed function with the distance map and fountains, and possibly using connectivity. Plot the result. Try to plot the grain boundaries on top once more.

In [ ]:
from skimage.segmentation import watershed
water = # use the watershed function

fig, ax = plt.subplots()
# plot the result using a colormap

### Exercise 14
Try to convert this segmented image into an RGB map and plot it with partial transparancy on top of the original image

## Measuring properties of the regions
Once we have a labeled image, it is easy to measure various properties

In [ ]:
from skimage.measure import regionprops_table, regionprops

In [ ]:
#to check the properties that can be measured
regionprops?

### Exercise 15
* Get the `label` and `area` metrics from the two segmented images (simple and complex segmentation) using `regionprops_table`. Inspect the returned objects. 
* Create images that map the area of each grain to each pixel.
* plot these images on top of the original image. Use subplots with the argument ncols = 2 to plot two maps in the same figure.

### Exercise 16
* Now plot histograms for the grain areas on the same plot. Make the histograms partially transparent.
* calculate the mean grain area and plot these as vertical lines on the plot
* add a label for all elements of the plot and add a legend

## Exporting data to excel
You can easily export the measurements to an excel or csv file

In [ ]:
import pandas as pd
df = pd.DataFrame(metrics)
df

In [ ]:
df.to_excel("grain_measurements.xlsx")

In [ ]:
!if [ -f grain_measurements.xlsx ];then echo file exists; fi